In [1]:
import pathlib

import numpy as np
from matplotlib import pyplot as plt
import scipy.spatial.transform as transform

import pycolmap
import pyvista as pv
import ipywidgets as widgets
from IPython.display import display
from utils import debounce

In [2]:
pv.set_jupyter_backend("trame")

In [3]:
image_dir: pathlib.Path = pathlib.Path("./images/south-building/")
working_dir: pathlib.Path = pathlib.Path("./temp/")
output_dir: pathlib.Path = pathlib.Path("./output/south-building/")

# Check path.
if not image_dir.exists():
    raise Exception("Input image directory doesn't exist.")
if not working_dir.exists():
    working_dir.mkdir(parents=True)
db_path = working_dir / "database.db"
if not output_dir.exists():
    output_dir.mkdir(parents=True)

In [4]:
# COLMAP pipeline
# pycolmap.extract_features(db_path, image_dir, camera_model="SIMPLE_PINHOLE")
# pycolmap.match_exhaustive(db_path)
# maps = pycolmap.incremental_mapping(db_path, image_dir, output_dir)[0]

# Load reconstruction cache.
maps = pycolmap.Reconstruction(output_dir / "0")
print(maps.summary())

Reconstruction:
	num_reg_images = 128
	num_cameras = 1
	num_points3D = 102877
	num_observations = 629479
	mean_track_length = 6.11875
	mean_observations_per_image = 4917.8
	mean_reprojection_error = 0.664317


In [5]:
num_points = len(maps.points3D)
points = np.empty((num_points, 3))
colors = np.empty((num_points, 3))

# Extract point cloud.
for i, point in enumerate(maps.points3D.values()):
    points[i] = point.xyz
    colors[i] = point.color
# Filter points within 10 units.
dists = np.linalg.norm(points, axis=1)
points = points[dists<=10, :]
colors = colors[dists<=10, :]
# Scale colors to [0, 1]
colors = colors / 256

points.shape, colors.shape

((102849, 3), (102849, 3))

In [6]:
images = maps.images.values()
num_images = len(images)
# Extract camera view.
cam_view_v = np.zeros((num_images * 5, 3))
cam_view_e = np.zeros((num_images * 8, 3), dtype=int)
cam_poss = np.zeros((num_images, 3))
cam_ups = np.zeros((num_images, 3))

for i, image in enumerate(images):
    cam_id = image.camera_id
    f, cx, cy = maps.cameras[cam_id].params
    dx = cx/f
    dy = cy/f
    # Get camera pose.
    cam_pos = image.cam_from_world.inverse().translation
    cam_rot = image.cam_from_world.inverse().rotation.quat
    rot_mat = transform.Rotation.from_quat(cam_rot).as_matrix()
    cam_z = np.matmul(rot_mat, np.array([0, 0, 1]))
    cam_y = np.matmul(rot_mat, np.array([0, -1, 0]))
    cam_x = np.matmul(rot_mat, np.array([-1, 0, 0]))
    cam_poss[i, :] = cam_pos
    cam_ups[i, :] = cam_y
    # Compose camera frame.
    cam_view_v[i*5 + 0, :] = cam_pos
    cam_view_v[i*5 + 1, :] = cam_pos + cam_z + cam_x * dx + cam_y * dy
    cam_view_v[i*5 + 2, :] = cam_pos + cam_z + cam_x * dx - cam_y * dy
    cam_view_v[i*5 + 3, :] = cam_pos + cam_z - cam_x * dx + cam_y * dy
    cam_view_v[i*5 + 4, :] = cam_pos + cam_z - cam_x * dx - cam_y * dy
    cam_view_e[i*8 + 0, :] = [2, i*5 + 0, i*5 + 1]
    cam_view_e[i*8 + 1, :] = [2, i*5 + 0, i*5 + 2]
    cam_view_e[i*8 + 2, :] = [2, i*5 + 0, i*5 + 3]
    cam_view_e[i*8 + 3, :] = [2, i*5 + 0, i*5 + 4]
    cam_view_e[i*8 + 4, :] = [2, i*5 + 1, i*5 + 2]
    cam_view_e[i*8 + 5, :] = [2, i*5 + 3, i*5 + 4]
    cam_view_e[i*8 + 6, :] = [2, i*5 + 1, i*5 + 3]
    cam_view_e[i*8 + 7, :] = [2, i*5 + 2, i*5 + 4]

In [7]:
pl = pv.Plotter()

def draw(cam_poss, cam_ups, cam_view_v, cam_view_e):
    pl.clear()
    # Plot point cloud.
    pl.add_points(
        points,
        scalars=colors,
        rgba=True
    )
    
    # Plot cameras.
    cam_points = pv.PolyData(cam_poss)
    cam_points["up"] = cam_ups
    cam_up_arrows = cam_points.glyph(
        orient="up",
        scale=False,
        factor=0.5
    )
    pl.add_mesh(cam_points, color="red", point_size=5)
    pl.add_mesh(cam_up_arrows, color="blue")
    # Plot camera frames.
    cam_mesh = pv.PolyData(cam_view_v, cam_view_e)
    pl.add_mesh(cam_mesh, color="red", style="wireframe")

draw(cam_poss, cam_ups, cam_view_v, cam_view_e)

# Camera control widgets.
cam_select_widget = widgets.Checkbox(
    value=False,
    description="Show Single Camera",
    disabled=False,
    indent=False
)
display(cam_select_widget)
cam_id_widget = widgets.IntSlider(
    min=0,
    max=num_images-1,
    description="Camera ID: ",
    disabled=True
)
display(cam_id_widget)
@debounce(0.5)
def on_cam_selected_change(change):
    cam_select = change["new"]
    cam_id_widget.disabled = not cam_select
    if cam_select:
        cam_id = cam_id_widget.value
        draw(
            cam_poss[cam_id, :].reshape((-1, 3)),
            cam_ups[cam_id, :].reshape((-1, 3)),
            cam_view_v[cam_id*5:(cam_id+1)*5, :].reshape((-1, 3)),
            np.array([
                [2, 0, 1],
                [2, 0, 2],
                [2, 0, 3],
                [2, 0, 4],
                [2, 1, 2],
                [2, 3, 4],
                [2, 1, 3],
                [2, 2, 4],
            ])
        )
    else:
        draw(cam_poss, cam_ups, cam_view_v, cam_view_e)
    pl.update()
@debounce(0.5)
def on_cam_id_change(change):
    cam_id = change["new"]
    draw(
        cam_poss[cam_id, :].reshape((-1, 3)),
        cam_ups[cam_id, :].reshape((-1, 3)),
        cam_view_v[cam_id*5:(cam_id+1)*5, :].reshape((-1, 3)),
            np.array([
                [2, 0, 1],
                [2, 0, 2],
                [2, 0, 3],
                [2, 0, 4],
                [2, 1, 2],
                [2, 3, 4],
                [2, 1, 3],
                [2, 2, 4],
            ])
    )
cam_select_widget.observe(on_cam_selected_change, names="value")
cam_id_widget.observe(on_cam_id_change, names="value")

pl.show()

Checkbox(value=False, description='Show Single Camera', indent=False)

IntSlider(value=0, description='Camera ID: ', disabled=True, max=127)

Widget(value='<iframe src="http://localhost:52864/index.html?ui=P_0x15e4bd710_0&reconnect=auto" class="pyvista…